# Get pictures for all MPs

In [185]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [186]:
# Read in details of MPs
mps = pd.read_hdf("list_of_mps.h5", "mps")

In [187]:
mps

,First name,Last name,Party,Constituency,URI,full_name,clean_name,is_female,party_women,party_mysoc,mp_wikidata,mp_wikidata_id,party_wikidata
Person ID,,,,,,,,,,,,,
10001,Diane,Abbott,Labour,Hackney North and Stoke Newington,https://www.theyworkforyou.com/mp/10001/diane_...,Diane Abbott,dianeabbott,True,Labour,Labour,Diane Abbott_1953,http://www.wikidata.org/entity/Q153454,Labour
10002,Gerry,Adams,Sinn Féin,Belfast West,https://www.theyworkforyou.com/mp/10002/gerry_...,Gerry Adams,gerryadams,False,None,Sinn Féin,Gerry Adams_1948,http://www.wikidata.org/entity/Q76139,Sinn Féin
10003,Irene,Adams,Labour,Paisley North,https://www.theyworkforyou.com/mp/10003/mrs_ir...,Irene Adams,ireneadams,True,Labour,Labour,"Irene Adams, Baroness Adams of Craigielea_1947",http://www.wikidata.org/entity/Q334498,Labour
10004,Nick,Ainger,Labour,Carmarthen West and South Pembrokeshire,https://www.theyworkforyou.com/mp/10004/nick_a...,Nick Ainger,nickainger,False,None,Labour,Nick Ainger_1949,http://www.wikidata.org/entity/Q325367,Labour
10005,Bob,Ainsworth,Labour,Coventry North East,https://www.theyworkforyou.com/mp/10005/bob_ai...,Bob Ainsworth,bobainsworth,False,None,Labour,Bob Ainsworth_1952,http://www.wikidata.org/entity/Q258738,Labour
10006,Peter,Ainsworth,Conservative,East Surrey,https://www.theyworkforyou.com/mp/10006/peter_...,Peter Ainsworth,peterainsworth,False,None,Conservative,Peter Ainsworth_1956,http://www.wikidata.org/entity/Q337709,Conservative
10007,Richard,Allan,Liberal Democrat,"Sheffield, Hallam",https://www.theyworkforyou.com/mp/10007/mr_ric...,Richard Allan,richardallan,False,None,Liberal Democrat,"Richard Allan, Baron Allan of Hallam_1966",http://www.wikidata.org/entity/Q334455,Liberal Democrat
10008,Graham,Allen,Labour,Nottingham North,https://www.theyworkforyou.com/mp/10008/graham...,Graham Allen,grahamallen,False,None,Labour,Graham Allen_1953,http://www.wikidata.org/entity/Q259601,Labour
10009,David,Amess,Conservative,Southend West,https://www.theyworkforyou.com/mp/10009/david_...,David Amess,davidamess,False,None,Conservative,David Amess_1952,http://www.wikidata.org/entity/Q259646,Conservative


In [188]:
content = BeautifulSoup(requests.get("https://beta.parliament.uk/houses/6GgqhSBt/members/current").text, "html5lib")

In [189]:
import urllib.request
import urllib.parse
from multiprocessing import Pool

In [190]:
mp_links = content.findAll("ul", {"class":"list--block"})[0].findAll("li")
mp_links = pd.DataFrame([{"name": mp.img["alt"],
                          "party": mp.findAll("p")[-1].text,
                          "constituency": mp.findAll("p")[0].text[len("MP for "):],
                          "imgref": mp.img["src"].split("/")[-1].split(".")[0]} for mp in mp_links])

mp_links = mp_links.query("name != 'placeholder'")[["name", "party", "constituency", "imgref"]]

In [191]:
mp_links

,name,party,constituency,imgref
0,Ms Diane Abbott,Labour,Hackney North and Stoke Newington,S3bGSTqn
2,Nigel Adams,Conservative,Selby and Ainsty,7xQEDGdb
3,Bim Afolami,Conservative,Hitchin and Harpenden,SrtAaMTP
4,Adam Afriyie,Conservative,Windsor,BfD4jWAV
5,Peter Aldous,Conservative,Waveney,G3znKiVx
6,Heidi Alexander,Labour,Lewisham East,qnsCGpnw
7,Rushanara Ali,Labour,Bethnal Green and Bow,tBnbaUGL
8,Lucy Allan,Conservative,Telford,DUPfVet1
9,Heidi Allen,Conservative,South Cambridgeshire,irqMbhPb
10,Dr Rosena Allin-Khan,Labour,Tooting,XWRJgHdJ


In [314]:
def get_mp_photo(mp_ref):
    urllib.request.urlretrieve("https://api-parliament-uk.azure-api.net/Live/photo/{0}.jpeg?crop=CU_1:1&amp;quality=90".format(mp_ref),
                   "./mp_photos/parl_{0}.jpg".format(mp_ref))
    return "./mp_photos/parl_{0}.jpg".format(mp_ref)

In [315]:
mp_links["imgref_parl"] = list(map(get_mp_photo, mp_links["imgref"].tolist()))

KeyboardInterrupt: 

In [316]:
mp_links

,name,party,constituency,imgref
0,Diane Abbott,Labour,Hackney North and Stoke Newington,S3bGSTqn
2,Nigel Adams,Conservative,Selby and Ainsty,7xQEDGdb
3,Bim Afolami,Conservative,Hitchin and Harpenden,SrtAaMTP
4,Adam Afriyie,Conservative,Windsor,BfD4jWAV
5,Peter Aldous,Conservative,Waveney,G3znKiVx
6,Heidi Alexander,Labour,Lewisham East,qnsCGpnw
7,Rushanara Ali,Labour,Bethnal Green and Bow,tBnbaUGL
8,Lucy Allan,Conservative,Telford,DUPfVet1
9,Heidi Allen,Conservative,South Cambridgeshire,irqMbhPb
10,Rosena Allin-Khan,Labour,Tooting,XWRJgHdJ


In [195]:
# Remove honorifics from names
import re
honorifics = r'(Mr|Mrs|Ms|Miss|Advocate|Ambassador|Baron|Baroness|Brigadier|Canon|Captain|Chancellor|Chief|Col|Comdr|Commodore|Councillor|Count|Countess|Dame|Dr|Duke of|Earl|Earl of|Father|General|Group Captain|H R H the Duchess of|H R H the Duke of|H R H The Princess|HE Mr|HE Senora|HE The French Ambassador M|His Highness|His Hon|His Hon Judge|Hon|Hon Ambassador|Hon Dr|Hon Lady|Hon Mrs|HRH|HRH Sultan Shah|HRH The|HRH The Prince|HRH The Princess|HSH Princess|HSH The Prince|Judge|King|Lady|Lord|Lord and Lady|Lord Justice|Lt Cdr|Lt Col|Madam|Madame|Maj|Maj Gen|Major|Marchesa|Marchese|Marchioness|Marchioness of|Marquess|Marquess of|Marquis|Marquise|Master|Mr and Mrs|Mr and The Hon Mrs|President|Prince|Princess|Princessin|Prof|Prof Emeritus|Prof Dame|Professor|Queen|Rabbi|Representative|Rev Canon|Rev Dr|Rev Mgr|Rev Preb|Reverend|Reverend Father|Right Rev|Rt Hon|Rt Hon Baroness|Rt Hon Lord|Rt Hon Sir|Rt Hon The Earl|Rt Hon Viscount|Senator|Sir|Sister|Sultan|The Baroness|The Countess|The Countess of|The Dowager Marchioness of|The Duchess|The Duchess of|The Duke of|The Earl of|The Hon|The Hon Mr|The Hon Mrs|The Hon Ms|The Hon Sir|The Lady|The Lord|The Marchioness of|The Princess|The Reverend|The Rt Hon|The Rt Hon Lord|The Rt Hon Sir|The Rt Hon The Lord|The Rt Hon the Viscount|The Rt Hon Viscount|The Venerable|The Very Rev Dr|Very Reverend|Viscondessa|Viscount|Viscount and Viscountess|Viscountess|W Baron|W/Cdr)'
h = re.compile(honorifics.replace("|", r" \b|\b"))

mp_links["name"] = mp_links["name"].str.replace(h, "")

In [197]:
# Join image dataframe with mps dataframe to get Person ID -> image table
mp_links_join = mp_links.join(mps.reset_index().set_index("full_name"), on="name")[["Person ID", "name", "party", "constituency", "imgref", "Party", "Constituency"]].set_index("name")

# For for all MPs which have multiple matches, choose the match where constituencies are the same
mp_links_join = mp_links_join[mp_links_join.groupby("name").count()["party"] == 1]\
    .append(mp_links_join.loc[mp_links_join.groupby("name").count()["party"] > 1].query('constituency==Constituency'))

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [198]:
# For the MPs that have been matched, create a lookup table
mp_image_lookup = mp_links_join.reset_index()[["Person ID", "imgref"]].dropna().set_index("Person ID")
# Make sure Person ID is an integer
mp_image_lookup.index = mp_image_lookup.index.astype(int)
mp_image_lookup.head()

,imgref
Person ID,
10001,S3bGSTqn
24878,7xQEDGdb
25661,SrtAaMTP
11929,BfD4jWAV
24904,G3znKiVx


In [202]:
mp_images = mps.join(mp_image_lookup)

In [303]:
def get_image_from_wiki(mp_id):
    """When supplied with a wikidata entity url, check if entity has an image associated
    and download that image to ./mp_photos/wiki_entityid.ext"""
    
    import requests
    import os
    import glob
    
    mp_id = mp_id.split("/")[-1]
    # First check if file already exists in ./mp_photos/
    if len(glob.glob("./mp_photos/wiki_{0}.*".format(mp_id))) > 0:
        return glob.glob("./mp_photos/wiki_{0}.*".format(mp_id))[0]
    else:
        try:
            photo_url = requests.get("https://www.wikidata.org/w/api.php?action=wbgetclaims&entity={0}&property=P18&format=json".format(mp_id))
            photo_url = photo_url.json()["claims"]["P18"][0]["mainsnak"]["datavalue"]["value"]
        
            media_url = requests.get("https://commons.wikimedia.org/w/api.php?action=query&prop=imageinfo&iiprop=url&titles=File:{0}&format=json".format(photo_url))
            media_url = list(media_url.json()["query"]["pages"].values())[0]["imageinfo"][0]["url"]

            # Finally, retrieve the image file, taking into account the correct file extension from photo_url
            filename = "./mp_photos/wiki_{0}.{1}".format(mp_id, photo_url.split(".")[-1])
            urllib.request.urlretrieve(media_url, filename)
            return filename
        except KeyError:
                return None
        

In [304]:
from multiprocessing import Pool
pool = Pool(8)
mp_filter = mp_images.imgref.isnull() & mp_images.mp_wikidata_id.notnull()
mp_images.loc[mp_filter, "imgref_wiki"] = pool.map(get_image_from_wiki, mp_images.loc[mp_filter]["mp_wikidata_id"].tolist())

pool.close()
pool.join()

In [317]:
mp_images.loc[mp_images.imgref.notnull() | mp_images.imgref_wiki.notnull()]

,First name,Last name,Party,Constituency,URI,full_name,clean_name,is_female,party_women,party_mysoc,mp_wikidata,mp_wikidata_id,party_wikidata,imgref,imgref_wiki
Person ID,,,,,,,,,,,,,,,
10001,Diane,Abbott,Labour,Hackney North and Stoke Newington,https://www.theyworkforyou.com/mp/10001/diane_...,Diane Abbott,dianeabbott,True,Labour,Labour,Diane Abbott_1953,http://www.wikidata.org/entity/Q153454,Labour,S3bGSTqn,NaN
10002,Gerry,Adams,Sinn Féin,Belfast West,https://www.theyworkforyou.com/mp/10002/gerry_...,Gerry Adams,gerryadams,False,None,Sinn Féin,Gerry Adams_1948,http://www.wikidata.org/entity/Q76139,Sinn Féin,NaN,./mp_photos/wiki_Q76139.jpg
10005,Bob,Ainsworth,Labour,Coventry North East,https://www.theyworkforyou.com/mp/10005/bob_ai...,Bob Ainsworth,bobainsworth,False,None,Labour,Bob Ainsworth_1952,http://www.wikidata.org/entity/Q258738,Labour,NaN,./mp_photos/wiki_Q258738.jpg
10009,David,Amess,Conservative,Southend West,https://www.theyworkforyou.com/mp/10009/david_...,David Amess,davidamess,False,None,Conservative,David Amess_1952,http://www.wikidata.org/entity/Q259646,Conservative,76klBZSN,NaN
10010,Michael,Ancram,Conservative,Devizes,https://www.theyworkforyou.com/mp/10010/michae...,Michael Ancram,michaelancram,False,None,Conservative,Michael Ancram_1945,http://www.wikidata.org/entity/Q332962,Conservative,NaN,./mp_photos/wiki_Q332962.jpg
10013,James,Arbuthnot,Conservative,North East Hampshire,https://www.theyworkforyou.com/mp/10013/james_...,James Arbuthnot,jamesarbuthnot,False,None,Conservative,James Arbuthnot_1952,http://www.wikidata.org/entity/Q259736,Conservative,NaN,./mp_photos/wiki_Q259736.jpg
10014,Hilary,Armstrong,Labour,North West Durham,https://www.theyworkforyou.com/mp/10014/hilary...,Hilary Armstrong,hilaryarmstrong,True,Labour,Labour,Hilary Armstrong_1945,http://www.wikidata.org/entity/Q332660,Labour,NaN,./mp_photos/wiki_Q332660.jpg
10015,Paddy,Ashdown,Liberal Democrat,Yeovil,https://www.theyworkforyou.com/mp/10015/mr_pad...,Paddy Ashdown,paddyashdown,False,None,Liberal Democrat,Paddy Ashdown_1941,http://www.wikidata.org/entity/Q333066,Liberal Democrat,NaN,./mp_photos/wiki_Q333066.jpg
10022,Norman,Baker,Liberal Democrat,Lewes,https://www.theyworkforyou.com/mp/10022/norman...,Norman Baker,normanbaker,False,None,Liberal Democrat,Norman Baker_1957,http://www.wikidata.org/entity/Q260082,Liberal Democrat,NaN,./mp_photos/wiki_Q260082.jpg
